9. Determine cual de todos los modelos es mejor, puede usar AIC y BIC para esto, además de
   los parámetros de la matriz de confusión y los del profilerb

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
from sklearn.linear_model import LogisticRegression
#Metrics
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import cProfile

In [14]:
# Seleccionar solo las variables numéricas relevantes
datos = pd.read_csv("train.csv", encoding="latin1")
numericas = datos.select_dtypes(include='number')
numericas = numericas.drop(["Fireplaces", "GarageYrBlt", "Id", "MSSubClass", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "MoSold", "YrSold"], axis=1)

numericas = numericas.dropna()

In [15]:
# Crear variable respuesta que clasifica las casas en Económicas, Intermedias o Caras
terciles = np.percentile(numericas["SalePrice"], [33.33, 66.67])
limite_economicas = terciles[0]
limite_caras = terciles[1]
numericas["Clase"] = pd.cut(numericas["SalePrice"], bins=[0, limite_economicas, limite_caras, float("inf")], labels=["Económicas", "Intermedias", "Caras"])

In [16]:
dummies = pd.get_dummies(numericas['Clase'])
dummies.head()

,Económicas,Intermedias,Caras
0,0,0,1
1,0,1,0
2,0,0,1
3,0,1,0
4,0,0,1


In [17]:
datos = pd.concat([numericas,dummies], axis=1)
datos.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,TotRmsAbvGrd,GarageCars,GarageArea,PoolArea,MiscVal,SalePrice,Clase,Económicas,Intermedias,Caras
0,65.0,8450,7,5,2003,2003,196.0,706,0,150,...,8,2,548,0,0,208500,Caras,0,0,1
1,80.0,9600,6,8,1976,1976,0.0,978,0,284,...,6,2,460,0,0,181500,Intermedias,0,1,0
2,68.0,11250,7,5,2001,2002,162.0,486,0,434,...,6,2,608,0,0,223500,Caras,0,0,1
3,60.0,9550,7,5,1915,1970,0.0,216,0,540,...,7,3,642,0,0,140000,Intermedias,0,1,0
4,84.0,14260,8,5,2000,2000,350.0,655,0,490,...,9,3,836,0,0,250000,Caras,0,0,1


In [22]:
X = datos
y = caras

random.seed(42)

In [24]:
X_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [25]:
logReg = LogisticRegression(solver='liblinear')
logReg.fit(X_train,y_train)
y_pred = logReg.predict(X_test)
cm = confusion_matrix(y_test,y_pred)

In [26]:
accuracy=accuracy_score(y_test,y_pred)
precision =precision_score(y_test, y_pred,average='micro')
recall =  recall_score(y_test, y_pred,average='micro')
f1 = f1_score(y_test,y_pred,average='micro')
print('Matriz de confusión para detectar si es cara o no\n',cm)
print('Accuracy: ',accuracy)
print('Precision: ', precision)
print('Recall score:', recall)
print('F1 score: ', f1)

Matriz de confusión para detectar si es cara o no
 [[154   2]
 [  1  82]]
Accuracy:  0.9874476987447699
Precision:  0.9874476987447699
Recall score: 0.9874476987447699
F1 score:  0.9874476987447699


In [27]:
k_folds = KFold(n_splits = 5, shuffle=True, random_state=42)

scores = cross_val_score(logReg, X, y, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.9874477 0.9958159 0.9916318 0.9958159 0.9916318]
Average CV Score:  0.992468619246862
Number of CV Scores used in Average:  5


In [28]:
# Stepwise regression
def forward_stepwise(X, y, initial_list=[], threshold_in=0.01, verbose=True):
    included = list(initial_list)
    excluded = [col for col in X.columns if col not in included]
    while True:
        changed = False
        excluded_dict = {}
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            p_value = model.pvalues[new_column]
            excluded_dict[new_column] = p_value
        best_column, best_pvalue = min(excluded_dict.items(), key=lambda x: x[1])
        if best_pvalue < threshold_in:
            included.append(best_column)
            excluded.remove(best_column)
            if verbose:
                print("Add {:30} with p-value {:.6}".format(best_column, best_pvalue))
            changed = True
        if not changed:
            break
    return included

In [29]:
selected_columns = forward_stepwise(X, y)

Add SalePrice                      with p-value 1.06628e-231
Add OverallQual                    with p-value 2.59376e-12
Add YearRemodAdd                   with p-value 7.30075e-07
Add OverallCond                    with p-value 1.41202e-07
Add FullBath                       with p-value 0.00174004
Add MasVnrArea                     with p-value 0.00257718
Add PoolArea                       with p-value 0.00571085
Add GarageArea                     with p-value 0.00650622
Add GarageCars                     with p-value 0.00623016


In [38]:
def entrenar_arbol(X_train, y_train, X_test, y_test, selected_columns):
    # Crear modelo de árbol de clasificación
    arbol = DecisionTreeClassifier(random_state=0)
    # Entrenar modelo
    arbol.fit(X_train, y_train)
    # Predecir valores para conjunto de prueba
    y_pred = arbol.predict(X_test)
    # Evaluar precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    # Creamos la matriz de confusion
    cm = confusion_matrix(y_test, y_pred)
    # Calculamos algunas métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Imprimimos las métricas de evaluación
    print("Exactitud: {:.2f}".format(accuracy))
    print("Precisión: {:.2f}".format(precision))
    print("Sensibilidad: {:.2f}".format(recall))
    print("Puntuación F1: {:.2f}".format(f1))
    
    # Cálculo de AIC y BIC
    n = len(X_test)
    k = len(selected_columns)
    log_likelihood = -n/2 * (1 + np.log(2*np.pi) + np.log(np.mean((y_test - y_pred)**2)))
    aic = -2*log_likelihood + 2*k
    bic = -2*log_likelihood + k*np.log(n)
    print("AIC: {:.2f}".format(aic))
    print("BIC: {:.2f}".format(bic))
    
    return arbol, cm


In [39]:
entrenar_arbol(X_train, y_train, X_test, y_test)

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00
AIC: -inf
BIC: -inf


C:\Users\estef\AppData\Local\Temp\ipykernel_44640\2697359233.py:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood = -n/2 * (1 + np.log(2*np.pi) + np.log(np.mean((y_test - y_pred)**2)))


(DecisionTreeClassifier(random_state=0),
 array([[156,   0],
        [  0,  83]], dtype=int64))

In [40]:
cProfile.run('entrenar_arbol(X_train, y_train, X_test, y_test)')

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00
AIC: -inf
BIC: -inf
         19851 function calls (19608 primitive calls) in 0.021 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.021    0.021 2697359233.py:1(entrenar_arbol)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
       13    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
       12    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
       16    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(any)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argmax)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(average)
        9    0.000    0.000    0.000   

C:\Users\estef\AppData\Local\Temp\ipykernel_44640\2697359233.py:26: RuntimeWarning: divide by zero encountered in log
  log_likelihood = -n/2 * (1 + np.log(2*np.pi) + np.log(np.mean((y_test - y_pred)**2)))


In [33]:
def random_forest_model(X_train, y_train, X_test, y_test, n_estimators=100, max_depth=None):
    # Crear el modelo de Random Forest con los parámetros deseados
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    # Entrenar el modelo con los datos de entrenamiento
    rf.fit(X_train, y_train)   
    # Predecir las clases para los datos de prueba
    y_pred = rf.predict(X_test)
    # Calcular precisión del modelo
    precision = accuracy_score(y_test, y_pred)
    # Creamos la matriz de confusion
    cm = confusion_matrix(y_test, y_pred)
    # Calculamos algunas métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    # Imprimimos las métricas de evaluación
    print("Exactitud: {:.2f}".format(accuracy))
    print("Precisión: {:.2f}".format(precision))
    print("Sensibilidad: {:.2f}".format(recall))
    print("Puntuación F1: {:.2f}".format(f1))

In [34]:
random_forest_model(X_train, y_train, X_test, y_test)

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00


In [35]:
cProfile.run('random_forest_model(X_train, y_train, X_test, y_test)')

Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00
         289090 function calls (287458 primitive calls) in 0.281 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.281    0.281 3932651776.py:1(random_forest_model)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(all)
      112    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(amax)
       12    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
      116    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(any)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argmax)
      102    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
       11    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(average)
      109    0.000    0.000    0.001    0.000 <__arr

In [41]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier

# Ajustar modelo de árbol de decisión
arbol = DecisionTreeClassifier(random_state=0)
arbol.fit(X_train, y_train)

# Predecir etiquetas para conjunto de prueba
y_pred = arbol.predict(X_test)

# Calcular matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Calcular métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Imprimir métricas de evaluación
print("Exactitud: {:.2f}".format(accuracy))
print("Precisión: {:.2f}".format(precision))
print("Sensibilidad: {:.2f}".format(recall))
print("Puntuación F1: {:.2f}".format(f1))


Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Puntuación F1: 1.00


10. Haga un modelo de árbol de decisión, uno de Random Forest y uno de Naive Bayes usando
    la misma variable respuesta y los mismos predictores que el mejor de los modelos de
    Regresión Logística

In [46]:
X = datos
y = caras

random.seed(42)

In [47]:
_train, X_test,y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

In [48]:
# Crear modelo de árbol de decisión
tree = DecisionTreeClassifier(random_state=42)

# Entrenar modelo
tree.fit(X_train, y_train)

# Predecir valores para conjunto de prueba
y_pred = tree.predict(X_test)

# Evaluar precisión del modelo
precision = accuracy_score(y_test, y_pred)

# Imprimir precisión del modelo
print("Precisión del modelo de árbol de decisión:", precision)


Precisión del modelo de árbol de decisión: 1.0


In [49]:
# Crear modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar modelo
rf.fit(X_train, y_train)

# Predecir valores para conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluar precisión del modelo
precision = accuracy_score(y_test, y_pred)

# Imprimir precisión del modelo
print("Precisión del modelo de Random Forest:", precision)


Precisión del modelo de Random Forest: 1.0


In [50]:
# Crear modelo de Naive Bayes
nb = GaussianNB()

# Entrenar modelo
nb.fit(X_train, y_train)

# Predecir valores para conjunto de prueba
y_pred = nb.predict(X_test)

# Evaluar precisión del modelo
precision = accuracy_score(y_test, y_pred)

# Imprimir precisión del modelo
print("Precisión del modelo de Naive Bayes:", precision)


Precisión del modelo de Naive Bayes: 0.9330543933054394


11. Compare la eficiencia de los 3 modelos que creó en el punto anterior y el mejor de los de
    regresión logística ¿Cuál se demoró más en procesar?¿Cuál se equivocó más?¿Cuál se
    equivocó menos?¿por qué?

In [52]:
# Importar módulos para modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time


In [53]:
# Crear modelo de regresión logística
log_reg = LogisticRegression(random_state=0)
start_time = time.time()
log_reg.fit(X_train, y_train)
end_time = time.time()
log_reg_time = end_time - start_time

# Predecir valores para conjunto de prueba utilizando modelo de regresión logística
y_pred = log_reg.predict(X_test)

# Calcular precisión del modelo de regresión logística
log_reg_accuracy = accuracy_score(y_test, y_pred)
log_reg_precision = precision_score(y_test, y_pred, average='weighted')
log_reg_recall = recall_score(y_test, y_pred, average='weighted')
log_reg_f1 = f1_score(y_test, y_pred, average='weighted')
log_reg_cm = confusion_matrix(y_test, y_pred)

C:\Users\estef\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [54]:
# Crear modelo de árbol de decisión
tree = DecisionTreeClassifier(random_state=0)
start_time = time.time()
tree.fit(X_train, y_train)
end_time = time.time()
tree_time = end_time - start_time

# Predecir valores para conjunto de prueba utilizando modelo de árbol de decisión
y_pred = tree.predict(X_test)

# Calcular precisión del modelo de árbol de decisión
tree_accuracy = accuracy_score(y_test, y_pred)
tree_precision = precision_score(y_test, y_pred, average='weighted')
tree_recall = recall_score(y_test, y_pred, average='weighted')
tree_f1 = f1_score(y_test, y_pred, average='weighted')
tree_cm = confusion_matrix(y_test, y_pred)


In [55]:
# Crear modelo de Random Forest
rf = RandomForestClassifier(random_state=0)
start_time = time.time()
rf.fit(X_train, y_train)
end_time = time.time()
rf_time = end_time - start_time

# Predecir valores para conjunto de prueba utilizando modelo de Random Forest
y_pred = rf.predict(X_test)

# Calcular precisión del modelo de Random Forest
rf_accuracy = accuracy_score(y_test, y_pred)
rf_precision = precision_score(y_test, y_pred, average='weighted')
rf_recall = recall_score(y_test, y_pred, average='weighted')
rf_f1 = f1_score(y_test, y_pred, average='weighted')
rf_cm = confusion_matrix(y_test, y_pred)


In [56]:
# Crear modelo de Naive Bayes
nb = GaussianNB()
start_time = time.time()
nb.fit(X_train, y_train)
end_time = time.time()
nb_time = end_time - start_time

# Predecir valores para conjunto de prueba utilizando modelo de Naive Bayes
y_pred = nb.predict(X_test)

In [61]:
# Calcular precisión del modelo de Naive Bayes
nb_accuracy = accuracy_score(y_test, y_pred)
nb_precision = precision_score(y_test, y_pred, average='weighted')
nb_recall = recall_score(y_test, y_pred, average='weighted')
nb_f1 = f1_score(y_test, y_pred, average='weighted')

print('Naive Bayes Model Metrics')
print('------------------------')
print('Accuracy:', nb_accuracy)
print('Precision:', nb_precision)
print('Recall:', nb_recall)
print('F1 Score:', nb_f1)


Naive Bayes Model Metrics
------------------------
Accuracy: 0.9330543933054394
Precision: 0.9335600653346162
Recall: 0.9330543933054394
F1 Score: 0.9332369254895911
